In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Cargar los datos de indicadores_economicos.csv
df_indicadores = pd.read_csv('indicadores_economicos.csv')

# Convertir la columna 'Year' a tipo fecha y extraer solo el año como entero
df_indicadores['Year'] = pd.to_datetime(df_indicadores['Year']).dt.year

# Columnas relevantes seleccionadas manualmente
columnas_relevantes = ['GDP (current US$) [NY.GDP.MKTP.CD]', 'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]', 'Gross capital formation (% of GDP) [NE.GDI.TOTL.ZS]', 
                       'Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]', 'Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]',
                       'Imports of goods and services (% of GDP) [NE.IMP.GNFS.ZS]', 'Total natural resources rents (% of GDP) [NY.GDP.TOTL.RT.ZS]']

# Agrupar por país
grupos_pais = df_indicadores.groupby('Country Name')

# Definir la función de ajuste para cada columna relevante
def ajustar_columna(datos):
    for columna in columnas_relevantes:
        # Obtener los valores conocidos y sus respectivos años
        x_known = datos['Year'].values.reshape(-1, 1)
        y_known = datos[columna].values
        
        # Encontrar índices de valores no NaN en 'y_known'
        idx_no_nan = ~np.isnan(y_known)
        
        # Continuar solo si hay al menos 3 puntos no NaN para ajustar un polinomio de grado 2
        if np.sum(idx_no_nan) >= 3:
            # Filtrar los valores no NaN para ajustar el modelo
            x_known_poly = x_known[idx_no_nan]
            y_known_poly = y_known[idx_no_nan]
            
            # Crear el modelo de regresión polinómica
            grado_polinomio = 2  # Puedes ajustar este valor al grado que desees
            poly_features = PolynomialFeatures(degree=grado_polinomio)
            x_known_poly = poly_features.fit_transform(x_known_poly)
        
            modelo_polinomico = LinearRegression()
        
            # Ajustar el modelo a los datos conocidos
            modelo_polinomico.fit(x_known_poly, y_known_poly)
            
            # Predecir los valores solo para los años con valores perdidos (NaN)
            x_pred = x_known[~idx_no_nan]  # Usar solo los años con NaN para predecir
            
            # Si no hay valores no NaN encontrados, entonces no hay necesidad de predecir
            if len(x_pred) == 0:
                continue
                
            x_pred_poly = poly_features.transform(x_pred)
            y_pred = modelo_polinomico.predict(x_pred_poly)
            
            # Actualizar los valores en el DataFrame original con los valores ajustados
            datos.loc[~idx_no_nan, columna] = y_pred  # Asignar valores predichos en los índices correspondientes
            
    return datos

# Aplicar la función de ajuste a cada grupo (país) y columna relevante
df_indicadores = grupos_pais.apply(ajustar_columna)

# Guardar el DataFrame con los valores ajustados en un nuevo archivo CSV
df_indicadores.to_csv('indicadores_economicos_ajustados_polinomio.csv', index=False)
